# Islamic Finance Ijarah MBT Processing Tool

This notebook implements a tool to analyze and process Ijarah Muntahia Bittamleek (Ijarah MBT) scenarios according to AAOIFI standards. It uses Retrieval-Augmented Generation (RAG) with LangChain to extract and process information from Islamic finance scenarios.

In [15]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


# Paths to your PDFs
pdf1_path = "../data/FAS32.pdf"
pdf2_path = "../data/SS9.pdf"

# Extract text
ijarah32_text = extract_pdf_text(pdf1_path)
ss9_text = extract_pdf_text(pdf2_path)

# Combine for RAG
all_docs = [
    {"source": "FAS32.pdf", "text": ijarah32_text},
    {"source": "SS9.pdf", "text": ss9_text}
]

# Chunk the text for retrieval
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
rag_chunks = []
for doc in all_docs:
    for chunk in splitter.split_text(doc["text"]):
        rag_chunks.append({"source": doc["source"], "text": chunk})

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

## 1. Data Extraction and Preparation

This cell extracts text from PDF documents containing AAOIFI standards relevant to Ijarah accounting. It:
- Uses pdfplumber to extract text from FAS32.pdf (Financial Accounting Standard 32) and SS9.pdf (Shariah Standard 9) documents
- Creates a structured format for the extracted text
- Splits the text into smaller chunks with some overlap for better retrieval

This prepares the data for creating a vector database to enable semantic search.

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv



# Define path for Chroma persistence
CHROMA_PATH = "../vector_db/MTB_ijarah_standards"
load_dotenv()

# Create embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Extract chunk texts and prepare metadata
chunk_texts = [chunk["text"] for chunk in rag_chunks]
chunk_metadatas = [{"source": chunk["source"]} for chunk in rag_chunks]

# Check if the vector store already exists
if os.path.exists(CHROMA_PATH):
    # Load existing vector store
    print("Loading existing Chroma vector store...")
    vector_store = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )
else:
    # Create a new vector store
    print("Creating new Chroma vector store...")
    vector_store = Chroma.from_texts(
        texts=chunk_texts,
        embedding=embeddings,
        metadatas=chunk_metadatas,
        persist_directory=CHROMA_PATH
    )
    # Persist the vector store to disk
    vector_store.persist()
    print(f"Saved vector store to {CHROMA_PATH}")
    
print(f"Vector store contains {vector_store._collection.count()} documents")

Loading existing Chroma vector store...
Vector store contains 486 documents


C:\Users\Tayeb Kahia\AppData\Local\Temp\ipykernel_23644\4121457770.py:23: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


## 2. Vector Database Creation

This cell creates a vector database using the Chroma vector store:
- Creates text embeddings using OpenAI's embedding model
- Either loads an existing vector store from disk or creates a new one
- Saves the vector store for future use

The vector database enables semantic search through the AAOIFI standards documents.

In [17]:
def retrieve_relevant_chunks(query, top_k=3):
    # Use Chroma's similarity search to find relevant chunks
    docs = vector_store.similarity_search(query, k=top_k)
    
    # Convert the returned documents to our expected format
    results = []
    for doc in docs:
        results.append({
            "source": doc.metadata["source"],
            "text": doc.page_content
        })
    return results


# Example: Retrieve relevant chunks for "Ijarah MBT Accounting"
relevant_chunks = retrieve_relevant_chunks(
    "Ijarah MBT Accounting in Lessee’s books")
for i, chunk in enumerate(relevant_chunks):
    print(
        f"--- Chunk {i+1} from {chunk['source']} ---\n{chunk['text'][:500]}\n")

--- Chunk 1 from FAS32.pdf ---
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38
Ijarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38
Accounting and financial reporting by the Lessor .......................................................................................... 39

--- Chunk 2 from FAS32.pdf ---
if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a
supplementary rental mechanism to recover the same, which shall be accounted for accordingly as a
revenue).
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah
BC27 The Board after taking into account industry comments agreed that the accounting for Ijarah MBT
through gradual transfer under FAS 8 was having practical impediments and required improvement.

--- Chunk 3 from FAS32.pdf ---
Ijarah MBT through gradual t

## 3. Relevant Document Retrieval

This cell defines a function to retrieve the most relevant text chunks based on a query:
- Uses semantic similarity search via the Chroma vector store
- Retrieves the top k most relevant chunks
- Formats the results for easy processing

The example query demonstrates retrieving information about Ijarah MBT accounting from the lessee's perspective.

In [18]:
import re


def calculate_ijarah_entries(params):
    prime_cost = params["purchase_price"] + \
        params["import_tax"] + params["freight"]
    rou = prime_cost - params["terminal_value"]
    total_rentals = params["yearly_rental"] * params["ijarah_term"]
    deferred_ijarah_cost = total_rentals - rou
    terminal_value_diff = params["residual_value"] - params["terminal_value"]
    amortizable_amount = rou - terminal_value_diff

    return {
        "prime_cost": prime_cost,
        "rou": rou,
        "total_rentals": total_rentals,
        "deferred_ijarah_cost": deferred_ijarah_cost,
        "terminal_value_diff": terminal_value_diff,
        "amortizable_amount": amortizable_amount
    }

## 4. Calculation Function

This cell defines a function to calculate the financial figures for Ijarah MBT accounting:
- Calculates the prime cost (purchase price + import tax + freight)
- Determines the Right-of-Use (ROU) asset value
- Calculates total rentals over the lease term
- Computes the deferred Ijarah cost
- Calculates terminal value difference and amortizable amount

These calculations follow AAOIFI's FAS 32 standard for Ijarah accounting.

In [19]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def extract_numbers_from_scenario_llm(scenario, llm=None):
    if llm is None:
        llm = OpenAI(temperature=0)

    prompt = PromptTemplate(
        input_variables=["scenario"],
        template="""
You are an expert Islamic finance accountant specializing in Ijarah MBT accounting according to AAOIFI standards.

The scenario describes an Ijarah Muntahia Bittamleek (Ijarah MBT) transaction that requires Initial Recognition at the time of commencement using the Underlying Asset Cost Method.

In this method:
- The right-of-use (ROU) asset is measured based on the cost of the asset to the lessor minus the terminal value
- Deferred Ijarah Cost is the difference between total rentals and the ROU value
- The Ijarah liability equals the total rentals to be paid over the term

Extract the following parameters from the scenario below. If a value is missing, return 0 or an appropriate default.

Scenario:
{scenario}

Return a JSON object with these keys:
- purchase_price (original cost of the asset to the lessor)
- import_tax (import duties or taxes paid)
- freight (shipping, installation, or transportation costs)
- ijarah_term (duration of the lease in years)
- residual_value (expected value at the end of the term)
- terminal_value (purchase option price for the lessee to acquire ownership)
- yearly_rental (annual payment amount)
- start_date (format: "D MMMM YYYY", like "1 January 2019")
- lessee_name (name of the institution that is the lessee)

Example output:
{{
  "purchase_price": 450000,
  "import_tax": 12000,
  "freight": 30000,
  "ijarah_term": 2,
  "residual_value": 5000,
  "terminal_value": 3000,
  "yearly_rental": 300000,
  "start_date": "1 January 2019",
  "lessee_name": "Alpha Islamic Bank"
}}
"""
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(scenario=scenario)

    # Parse the JSON from the LLM output
    import json
    import re
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        return json.loads(match.group(0))
    else:
        raise ValueError("Could not parse LLM output as JSON:\n" + response)

## 5. Parameter Extraction using LLM

This cell defines a function that uses an LLM (OpenAI) to extract numerical parameters from a text scenario:
- Creates a prompt template with detailed instructions about Ijarah MBT accounting
- The LLM extracts specific parameters like purchase price, taxes, rental amounts, etc.
- Returns a structured JSON object with all the required parameters

This automates the extraction of key financial figures from natural language case descriptions.

In [23]:
def ijarah_agent(scenario, query="Ijarah MBT Accounting in Lessee's books", top_k=3):
    # 1. Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(query, top_k=top_k)
    print("Relevant Chunks from PDFs:")
    for i, chunk in enumerate(relevant_chunks):
        print(
            f"--- Chunk {i+1} from {chunk['source']} ---\n{chunk['text'][:300]}...\n")

    # 2. Extract numbers from scenario
    params = extract_numbers_from_scenario_llm(scenario)
    print("Extracted Parameters:", params)

    # 3. Calculate entries
    results = calculate_ijarah_entries(params)
    
    # Format output as requested
    print("\nCorrect Solution:")
    start_date = params.get("start_date", "1 January 2019")  # Use extracted date or default
    lessee_name = params.get("lessee_name", "the Lessee")    # Use extracted lessee name or default
    
    print(f"Initial Recognition at the Time of Commencement of Ijarah ({start_date}):")
    print("The cost of the underlying asset is the basis for recognizing the \"Right of Use\" (ROU) asset.")
    print("(Using Underlying Asset Cost Method as per AAOIFI FAS 32)")
    
    print("\nDetermine the Right-of-Use Asset (ROU)")
    print("• Prime cost (Purchase + Import tax + Freight):")
    print(f"= {params['purchase_price']:,} + {params['import_tax']:,} + {params['freight']:,} = {results['prime_cost']:,}")
    print("• Less: Terminal value (i.e., Purchase price of USD 3,000 to acquire ownership)")
    print(f"= {results['prime_cost']:,} − {params['terminal_value']:,} = {results['rou']:,} (ROU)")
    
    print("\nAdd Deferred Ijarah Cost:")
    print(f"• Total rentals over {params['ijarah_term']} years = {params['yearly_rental']:,} × {params['ijarah_term']} = {results['total_rentals']:,}")
    print(f"• Less: Present value of ROU = {results['rou']:,}")
    print(f"• Deferred Ijarah Cost = {results['total_rentals']:,} − {results['rou']:,} = {results['deferred_ijarah_cost']:,}")
    
    print("\nJournal Entry:")
    print(f"Dr. Right of Use Asset (ROU)                USD {results['rou']:,}")
    print(f"Dr. Deferred Ijarah Cost                    USD {results['deferred_ijarah_cost']:,}")
    print(f"    Cr. Ijarah Liability                               USD {results['total_rentals']:,}")
    
    print("\nAmortizable Amount Calculation")
    print("Description                                           USD")
    print(f"Cost of ROU                                          {results['rou']:,}")
    print(f"Less: Terminal value difference (Residual {params['residual_value']:,} − Purchase {params['terminal_value']:,})    {results['terminal_value_diff']:,}")
    print(f"Amortizable Amount                                   {results['amortizable_amount']:,}")
    print(f"We deduct this {results['terminal_value_diff']:,} since the Lessee is expected to gain ownership, and this value will remain")
    print("in the books after the lease ends.")

    return results, relevant_chunks


# Example usage:
scenario = """
Ijarah MBT Accounting (in Lessee’s books)  
On 1 January 2019 Alpha Islamic bank (Lessee) entered into an Ijarah MBT arrangement with 
Super Generators for Ijarah of a heavy-duty generator purchase by Super Generators at a price 
of USD 450,000. 
Super Generators has also paid USD 12,000 as import tax and US 30,000 for freight charges. 
The Ijarah Term is 02 years and expected residual value at the end USD 5,000. At the end of 
Ijarah Term, it is highly likely that the option of transfer of ownership of the underlying asset to 
the lessee shall be exercised through purchase at a price of USD 3,000. 
Alpha Islamic Bank will amortize the ‘right of use’ on yearly basis and it is required to pay yearly 
rental of USD 300,000.  
Provide the following accounting entry in the books of Alpha Islamic Bank: 
Initial Recognition at the time of commencement of Ijarah (using Underlying Asset Cost 
Method)
"""

ijarah_agent(scenario)

Relevant Chunks from PDFs:
--- Chunk 1 from FAS32.pdf ---
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38
Ijarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38
Accounting and financial reporting by the ...

--- Chunk 2 from FAS32.pdf ---
if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a
supplementary rental mechanism to recover the same, which shall be accounted for accordingly as a
revenue).
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah
BC27 The Board after ...

--- Chunk 3 from FAS32.pdf ---
Ijarah MBT through gradual transfer – special considerations ................................................................. 28
De-recognition in the books of the seller / transferor (previously lessor) ..................................................... 28
Ijarah M

({'prime_cost': 492000,
  'rou': 489000,
  'total_rentals': 600000,
  'deferred_ijarah_cost': 111000,
  'terminal_value_diff': 2000,
  'amortizable_amount': 487000},
 [{'source': 'FAS32.pdf',
   'text': 'Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38\nIjarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38\nAccounting and financial reporting by the Lessor .......................................................................................... 39'},
  {'source': 'FAS32.pdf',
   'text': 'if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a\nsupplementary rental mechanism to recover the same, which shall be accounted for accordingly as a\nrevenue).\nIjarah MBT through gradual transfer / Diminishing Musharaka Ijarah\nBC27 The Board after taking into account industry comments agreed tha

## 6. Main Ijarah Agent Function

This cell defines the main Ijarah agent function that brings all components together to process a scenario:
1. Retrieves relevant content from the AAOIFI standards documents
2. Extracts parameters from the input scenario using the LLM
3. Calculates all the required financial figures
4. Formats and presents a complete accounting solution

The formatted output includes:
- Initial recognition entries
- Right-of-Use asset calculation
- Deferred Ijarah cost calculation
- Journal entries
- Amortizable amount calculation

The function is demonstrated with a sample Ijarah MBT scenario for Alpha Islamic Bank.

# Another Scenario use case 

In [24]:
# Test scenario with explicit reference to the Underlying Asset Cost Method
test_scenario = """
On June 1, 2023, Gulf Islamic Finance (Lessee) entered into an Ijarah Muntahia Bittamleek (MBT) arrangement with 
Premium Equipment Suppliers for a commercial printing machine. 

The machine was purchased by Premium Equipment Suppliers at a price of USD 320,000.
Premium Equipment Suppliers paid USD 18,000 for import duties and USD 22,000 for shipping and installation.

The Ijarah term is 4 years with the machine expected to have a residual value of USD 40,000 at the end of this period.
Gulf Islamic Finance has the option to purchase the machine for USD 25,000 at the end of the term, which they intend to exercise.

Gulf Islamic Finance will pay annual rentals of USD 110,000 and will amortize the right-of-use asset on a yearly basis.

Provide the accounting entry in Gulf Islamic Finance's books for the initial recognition at the time of commencement 
of the Ijarah using the Underlying Asset Cost Method as specified in AAOIFI FAS 32.
"""

# Run the agent with the test scenario
print("TESTING THE UNDERLYING ASSET COST METHOD WITH A NEW SCENARIO:\n")
ijarah_agent(test_scenario)

TESTING THE UNDERLYING ASSET COST METHOD WITH A NEW SCENARIO:

Relevant Chunks from PDFs:
--- Chunk 1 from FAS32.pdf ---
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38
Ijarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38
Accounting and financial reporting by the ...

--- Chunk 2 from FAS32.pdf ---
if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a
supplementary rental mechanism to recover the same, which shall be accounted for accordingly as a
revenue).
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah
BC27 The Board after ...

--- Chunk 3 from FAS32.pdf ---
Ijarah MBT through gradual transfer – special considerations ................................................................. 28
De-recognition in the books of the seller / transferor (previously lessor) ..

({'prime_cost': 360000,
  'rou': 335000,
  'total_rentals': 440000,
  'deferred_ijarah_cost': 105000,
  'terminal_value_diff': 15000,
  'amortizable_amount': 320000},
 [{'source': 'FAS32.pdf',
   'text': 'Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38\nIjarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38\nAccounting and financial reporting by the Lessor .......................................................................................... 39'},
  {'source': 'FAS32.pdf',
   'text': 'if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a\nsupplementary rental mechanism to recover the same, which shall be accounted for accordingly as a\nrevenue).\nIjarah MBT through gradual transfer / Diminishing Musharaka Ijarah\nBC27 The Board after taking into account industry comments agreed th

## 7. Test Case - Additional Scenario

This cell tests the Ijarah agent with a different scenario:
- Creates a new test case for Gulf Islamic Finance with a commercial printing machine
- Includes different financial parameters (cost, term, rental amounts, etc.)
- Processes the scenario through the same Ijarah agent function

This demonstrates how the tool can handle various Ijarah MBT scenarios while applying consistent AAOIFI standards.